In [ ]:
#!/usr/bin/python
import sys
import pickle
import os
sys.path.append("../tools/")

In [ ]:
from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from collections import defaultdict
%matplotlib inline

# Funções definidas

In [ ]:
def F_RetirarNaNs(data_dict):
    '''
    Retira os NaNs 
    data_dict: dicionário com diversas features para cada pessoa
    return: dicionário de entrada, exceto as features que possuem valor NaN
    '''
    
    for k1, v1 in data_dict.iteritems():
        data_dict[k1] = {key:value for key, value in v1.iteritems() if value != 'NaN'}
    
    return data_dict

In [ ]:
def F_FeatureScalling(data_dict, not_scale, features):
    '''
    Cria nova escala para features de cada pessoa no dicionário de entrada
    data_dict: dicionário com diversas features para cada pessoa
    not_scale: features que não devem ser alteradas
    return: tuple com features do dicionário em nova escala e um dicionário com o conversor de escalas
    '''
    
    from sklearn.preprocessing import MinMaxScaler
    
    #Criar escala para cada feature
    scale_feature = dict()
    for feature in features: 
        if not feature in not_scale:
            scale_feature[feature] = MinMaxScaler()
            scale_feature[feature].fit_transform(np.array([data_dict[key][feature]
                                                for key in data_dict.keys()
                                                if feature in data_dict[key].keys()]).reshape(-1,1))
    
    #Aplicar escala sobre os campos de interesse
    for k1, v1 in data_dict.iteritems():
        for key, value in v1.iteritems():
            if key in features and not key in not_scale:
                data_dict[k1][key] = scale_feature[key].transform(np.array(value).reshape(-1,1))[0][0]
            else:
                data_dict[k1][key] = value
    
    return data_dict, scale_feature

In [ ]:
def F_OrdemFeatures(data_dict, features):
    '''
    Retorna uma lista com as features ordenadas por valor
    data_dict: dicionários com as features
    features: features a serem ordenadas
    return: lista com ordem das features
    '''
    
    ordem = {}
    
    for feature in features:
        
        aux = {keys: values[feature] for keys, values in data_dict.items() if feature in values.keys()}
        
        ordem[feature] = sorted(aux.keys(), key=lambda x:data_dict[x][feature])
        
    return ordem

In [ ]:
def F_PlotOutliers(test_list, ordem):
    '''
    Criar gráfico com valores ordenados
    test_list: lista com os dados
    ordem: ordem dos gráficos
    return: maiores valores em cada gráfico (análise de outliers)
    '''
    
    outliers_dict = {}
    n_dados = len(test_list)
    
    f, gp = plt.subplots(n_dados,1, sharex= True)

    f.set_figheight((2.5*n_dados)//2+1)
    f.set_figwidth(15)

    for jj, feature in enumerate(test_list):
        max_value = 0
        person = ''
        
        try:

            for ii, point in enumerate(ordem[feature]):
                if feature in data_dict[point]:

                    salary = data_dict[point][feature]

                    if max_value < salary:
                        max_value = salary
                        person = point

                    #Registrando os valores extremos encontrados
                    outliers_dict[feature] = [max_value, person]

                    gp[jj].scatter(ii, salary)

                gp[jj].set_title(feature)
        except:
            pass

    plt.show()
    
    return outliers_dict

In [ ]:
def F_PessoaEmail(dir_, address):
    
    '''
    Procura no repositório dir_ o endereço de destino contido no email
    dir_: diretório dos emails
    address: email procurado
    '''
    lista = []
    if email in address.keys():
        with open('{}/{}'.format(dir_, address[email], 'r')) as fl:
            lista = [line.split('/')[2] for line in fl]
    
    return lista

In [ ]:
def F_ExtraiPoisAbreviados(data_dict):
    '''
    Extrair os POis e abrevia os nomes pela regra de sobrenome-Nome(primeira letra), exemplo, Kenneth Lee Lay para lay-k
    data_dict: dicionário com dados das pessoas
    return: dictionary com nomes convertidos por pessoa
    '''

    aux_list = {}
    for ii in [key for key, value in data_dict.items() if value['poi']==True]:
        string = ii.split(' ')
        name = string[0]

        second_name = string[1][0] if len(string) in [3,2] else string[2][0]
        
        try:
            aux_list[data_dict[ii]['email_address']] = '-'.join([name, second_name]).lower()
        except:
            print 'Erro na chave {key}'.format(key=ii)
        
    return aux_list

In [ ]:
def F_StatMsg(email, aux_list):
    '''
    Retorna a quantidade de e-mails trocados com POis
    email: dictionary com os destinos dos emails enviados por uma pessoa
    return: dictionary com contagem do total de emails enviados para POis para cada indivíduo
    '''
    
    count, ratio = {}, {}
    for key in email.keys():
        
        aux = aux_list.copy()
        if key in aux.keys():
        #Não contar a própria pessoa -- acontece quando a pessoa é um poi
            aux.pop(key)
        
        #Somente os que são poi
        lista = [ii for ii in email[key] if ii in aux.values()]
        
        #total de poi com troca de mensagens
        count[key] = len(set(lista))
        
        #razao entre mensagens com poi e total de mensagens
        if len(email[key])>0:
            ratio[key] = len(lista)/float(len(email[key]))
        else:
            ratio[key]= 0
    
    return count, ratio

In [ ]:
def F_VerificarExisteFeature():
    '''Verificar se existe chave sem alguma característica 
        return: key e feature faltante, além da quantidade de keys que precisam ser corrigidas
    '''
    
    person_feature = defaultdict(list)
    
    all_features = {key for key, value in my_dataset.items() if all(ii == features_list for ii in value.keys())}
    all_persons = {key for key in my_dataset.keys()}
    
    #procura as pessoas que não tem todas as features e encontra a features faltante
    for jj in all_persons - all_features:
        for ii in features_list:
            if not ii in my_dataset[jj].keys():
                person_feature[jj].append(ii)
    
    print 'Quantidade de pessoas com falta de informações, ', len(person_feature)
    return person_feature

In [ ]:
### Task 1: Select what features you'll use.
### features_list is a list of strings, each of which is a feature name.
### The first feature must be "poi".
features_list = ['poi','salary','total_payments', 'bonus', 'total_stock_value', 'expenses', 
                 'exercised_stock_options', 'restricted_stock','shared_receipt_with_poi','ratio_to','ratio_from'] 

In [ ]:
### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

# Cleaning Data
Temos que alguns campos estão como 'NaN', logo estes campos não serão considerados neste momento para facilitar a análise de outliers e outras análises que a falta de dados pode enviesar. Se os campos com 'NaN' estiverem na feature_list eles serão futuramente adicionados com valor 0.

Antes de retirar os valores NaN

In [ ]:
data_dict['METTS MARK']

In [ ]:
data_dict = F_RetirarNaNs(data_dict)

Após ser retirado os NaNs

In [ ]:
data_dict['METTS MARK']

A pasta email_by_address será utilizada nesse trabalho. Como algumas keys em data_dict não possuem e-mail, essas pessoas serão desconsideradas. Isso reduz a base de dados, mas trará maior consistência do que supor que essas não trocam e-mails.

In [ ]:
for key in data_dict.keys():
    if not 'email_address' in data_dict[key].keys():
        data_dict.pop(key)

# Task 2: Remove outliers
A análise gráfico será utilizada para descobrir se existem outliers na base de dados.

In [ ]:
plot_list = ['salary','total_payments', 'bonus', 'total_stock_value', 'expenses', 'exercised_stock_options', 
             'restricted_stock', 'from_poi_to_this_person', 'from_this_person_to_poi', 'shared_receipt_with_poi']
ordem = F_OrdemFeatures(data_dict, plot_list)
F_PlotOutliers(plot_list, ordem)

Temos que os valores extremos nos gráficos estão a pessoal com altos cargos na Enron, como:

**LAVORATO JOHN J**: Principais executivos

**LAY KENNETH L**: fundador e CEO da companhia na época do escândalo

**SKILLING JEFFREY K**: fundador e ex-CEO da companhia na época do escândalo

Essas pessoas são chefes da companhia, assim possuem salários e bonificações diferenciadas dos demais funcionários.

*É necessário analisar os outros pontos que podem ser outleirs para determinar se também se tratam de pessoas com altos cargos.*

### SALARY

In [ ]:
ordem['salary'][-3:]

Os pontos que poderiam ser outliers são de chefes da companhia. Nesse caso temos que além de **LAY KENNETH L** e **SKILLING JEFFREY K** temos **FREVERT MARK A** que era na época CEO de uma subsidiára do grupo a _Enron Whosale Services_

### EXERCISED STOCK OPTIONS

In [ ]:
ordem['exercised_stock_options'][-2:]

Para exercised_stock_options 2 altos executivos.

### RESTRICTED STOCK

In [ ]:
ordem['restricted_stock'][-3:]

No caso de restricted stock temos pessoas com altos cargos que recebiam remuneração através de ações associadas a desempenho. 

Temos que as 3 pessoas são altos executivos o que justifica os altos ganhos.

### EMAILS

A quantidade de emails pode variar muito independente da pessoa e por isso não será considerada um outlier os valores extremos no gráfico.

# Conclusão

Após o cleaning data não existem outliers.

# Task 3 --  Create New Features

Utilizando os dados contidos em email_by_address vou criar duas features que trazem a quantidade de POis que uma pessoa se relacionou enviando ou recebendo emails.

In [ ]:
#Para cada pessoa em data_dict pego os destinatários dos emails from e to

email_poi = ((data_dict[key]['email_address'], data_dict[key]['poi']) for key in data_dict.keys() 
                if 'email_address' in data_dict[key] and 'poi' in data_dict[key])

address_from = {file_.split('_')[1][:-4]:file_ for file_ in os.listdir('emails_by_address') if file_.split('_')[0]=='from'}
address_to = {file_.split('_')[1][:-4]:file_ for file_ in os.listdir('emails_by_address') if file_.split('_')[0]=='to'}

from_, to_ = {}, {}
for email, poi in email_poi:
    from_[email] = F_PessoaEmail('emails_by_address', address_from)
    to_[email] = F_PessoaEmail('emails_by_address', address_to)      

In [ ]:
poi_aux_list = F_ExtraiPoisAbreviados(data_dict)

In [ ]:
poi_aux_list

Determinar a quantidade POis com que a pessoa interage por e-mails

In [ ]:
#from_count, from_ratio = F_StatMsg(from_, poi_aux_list)
to_count, to_ratio = F_StatMsg(to_, poi_aux_list)

Aqui tenho as novas características construídas, preciso adicionar ao data_dict

from_ratio e to_ratio não serão incluídos, pois existem os campos from_messages, from_poi_to_this_person, from_this_person_to_poi e to_messages que podem ser combinados para gerar uma feature

In [ ]:
for key in data_dict.keys():
    
    email = data_dict[key]['email_address']
    
    data_dict[key]['from_count'] = from_count[email]
    data_dict[key]['to_count'] = to_count[email]

### Adicionando razao de envio de e-mail
As pessoas que não possuem algum campo de mensagem serão excluídas da base para manter a consistência. O outro caminho seria considerar como sendo 0 os campos de mensagens, mas prefiro nesse caso manter a consistências das informações a supor um valor.

In [ ]:
data_dict[data_dict.keys()[27]]

In [ ]:
for key, value in data_dict.items():
    try:
        
        if value['to_messages'] == 0:
            value['ratio_to'] = 0
        
        if value['from_messages'] == 0:
            value['ratio_from'] = 0 
        
        if not 0 in [value['to_messages'],value['from_messages']]:
            value['ratio_to'] = value['from_this_person_to_poi'] / float(value['to_messages'])
            value['ratio_from'] = value['from_poi_to_this_person'] / float(value['from_messages'])
            
        value.pop('from_this_person_to_poi')
        value.pop('to_messages')
        value.pop('from_poi_to_this_person')
        value.pop('from_messages')
            
        
    except:
        #se o campo não existe, não considero na base final
        data_dict.pop(key)

A base de dados está reduzida devido a desconsiderar as keys em data_dict que não possuem o campo e-mail

In [ ]:
len(data_dict)

In [ ]:
data_dict[data_dict.keys()[9]]

# Feature Scalling
Para evitar erros computacionais os valores devem ser bem condicionados, logo vou deixar os dados entre 0 e 1

In [ ]:
features_list = ['poi','salary','total_payments', 'bonus', 'total_stock_value', 'expenses', 
                 'exercised_stock_options', 'restricted_stock',
                 'email_address', 'shared_receipt_with_poi', 'to_count', 'from_count','ratio_to','ratio_from'] 

In [ ]:
data_dict['METTS MARK']

In [ ]:
data_dict, scale  = F_FeatureScalling(data_dict, ['email_address','poi'], features_list)

In [ ]:
data_dict['METTS MARK']

Com isso termino de fazer as correções da base e mudo a escala dos dados para fazer para facilitar os cáclulos numéricos

In [ ]:
### Task 3: Create new feature(s)
### Store to my_dataset for easy export below.
my_dataset = data_dict

# Corrigindo a base

Verificando se todas as keys em data_dict possuem todas as features em features_list

In [ ]:
not_person_feature = F_VerificarExisteFeature()
not_person_feature

Para não reduzir ainda mais a base de dados, vou adicionar as features em features_list em todas as keys contidas em data_dict para que seja possível gerar o numpy array

In [ ]:
for key, values in not_person_feature.items():
    for value in values:
        my_dataset[key][value] = 0

Verificando novamente se existe algum campo com problema

In [ ]:
F_VerificarExisteFeature()

# Extração de características

Necessário retirar email_address dos campos de características

In [ ]:
features_validas = features_list[:]
features_validas.remove('email_address')

# Extrair features e labels

In [ ]:
### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_validas, sort_keys = True)
labels, features = targetFeatureSplit(data)

# Task 4: Try a varity of classifiers

In [ ]:
### Task 4: Try a varity of classifiers
### Please name your classifier clf for easy export below.
### Note that if you want to do PCA or other multi-stage operations,
### you'll need to use Pipelines. For more info:
### http://scikit-learn.org/stable/modules/pipeline.html

# Provided to give you a starting point. Try a variety of classifiers.
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
### Task 5: Tune your classifier to achieve better than .3 precision and recall 
### using our testing script. Check the tester.py script in the final project
### folder for details on the evaluation method, especially the test_classifier
### function. Because of the small size of the dataset, the script uses
### stratified shuffle split cross validation. For more info: 
### http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedShuffleSplit.html

# Example starting point. Try investigating other evaluation techniques!
from sklearn.cross_validation import train_test_split
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

### Importando função para teste

In [ ]:
from tester import test_classifier

### Classifier Naive Bayes

In [ ]:
clf = GaussianNB()
clf.fit(features_train, labels_train)
clf.score(features_test,labels_test)

In [ ]:
test_classifier(clf, my_dataset, features_validas)

### SVM

In [ ]:
clf = SVC(C=100, kernel='rbf', gamma=10)
clf.fit(features_train, labels_train)
clf.score(features_test,labels_test)

In [ ]:
test_classifier(clf, my_dataset, features_validas)

### Neural Nets

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(1,),solver='lbfgs', learning_rate_init =.1)
clf.fit(features_train, labels_train)
clf.score(features_test,labels_test)

In [ ]:
test_classifier(clf, my_dataset, features_validas)

### Decision Trees

In [ ]:
clf = DecisionTreeClassifier(criterion='gini', min_samples_split=3, max_depth=2,splitter='random')
clf.fit(features_train, labels_train)
clf.score(features_test,labels_test)

In [ ]:
test_classifier(clf, my_dataset, features_validas)

### Gradiente Boosting

In [ ]:
clf = GradientBoostingClassifier()
clf.fit(features_train, labels_train)
clf.score(features_test,labels_test)

In [ ]:
test_classifier(clf, my_dataset, features_validas)

### Classifier Random Forest

In [ ]:
clf = RandomForestClassifier(criterion='entropy', max_features='log2',warm_start =True)
clf.fit(features_train, labels_train)
clf.score(features_test,labels_test)

In [ ]:
test_classifier(clf, my_dataset, features_validas)

O melhor classificador foi o Random Forest. Devido a pequena quantidade de dados os classificadores testados tem uma grande de chance de sofrer Overfitting. Nos testes encontramos que Random Forest se adaptou melhor a pequena quantidade de dados por ter sua resposta baseado em vários classificadores fracos que não sofrem tanto com overfitting.

In [ ]:
### Task 6: Dump your classifier, dataset, and features_list so anyone can
### check your results. You do not need to change anything below, but make sure
### that the version of poi_id.py that you submit can be run on its own and
### generates the necessary .pkl files for validating your results.

dump_classifier_and_data(clf, my_dataset, features_validas)